In [ ]:
%matplotlib nbagg
#from photodiag import PalmSetup
import photodiag
import json
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import h5py
import os, glob


from scipy.special import erf
from scipy.optimize import curve_fit
from alvra_tools import load_data
from IPython.display import clear_output

# First part: load a THz scan to calibrate the eV to fs slope

### First look for the correct PALM calibration file

In [ ]:
#DIRcalib = '/sf/photo/src/PALM/calib/'
#DIRcalib = '/sf/photo/src/PALM/calib/Alvra'
#DIRcalib = '/sf/alvra/data/p18442/res/PhotoDiag/scan_info/'
DIRcalib = '/sf/alvra/data/p18182/res/PhotoDiag/scan_info/'

!ls -lah -rt $DIRcalib | grep palm_etof

### Now load the calibration file and give the energy range

In [ ]:
#CalibrationFn = DIRcalib + '2019-05-24_13:40:42.palm_etof'     # 12 keV settings
CalibrationFn = DIRcalib + '2020-01-25_16:28:32.palm_etof'     # 5 keV settings -- p18182

energyFrom =  8400
energyTo =    8600
energySteps = 2000

###########################################################################################

palm = photodiag.PalmSetup({'0': 'SAROP11-PALMK118:CH1_BUFFER', '1': 'SAROP11-PALMK118:CH2_BUFFER'},
                 noise_range=[0, 250],
                 energy_range=np.linspace(energyFrom, energyTo, energySteps))

palm.load_etof_calib(CalibrationFn)    

# define the fitfunction
def fitThz(x, a, b, c, d, e, f):
    return a + b*np.exp(-(c-x)**2/np.abs(d)**2)*np.sin(e*x + f) 


### Now choose the directory of the json file for the THz scan

In [ ]:
DIRjson = "/sf/alvra/data/p18182/res/PhotoDiag/scan_info/"

!ls -lah -rt $DIRjson | grep json

 ### Load the THz scan

In [ ]:
scan_name = 'PALM_scan_508'

###########################################

fn_json = DIRjson + scan_name + '_scan_info.json'

with open(fn_json) as f:
    dataFiles = json.load(f)
numFiles = len(dataFiles['scan_files'])
StagePOS = dataFiles['scan_values'][:]

eVIntP = []
eVIntUp = []
wf_str = []
wf_ref = []

for i in range(0,numFiles):
    fn = str(dataFiles['scan_files'][i][0])
    with h5py.File(fn, 'r') as fileName:
        fileName = load_data._get_data(fileName)
        
        TOF = -fileName['SAROP11-PALMK118:CH2_BUFFER/data'][:]
        uTOF = -fileName['SAROP11-PALMK118:CH1_BUFFER/data'][:]
        PulseIds = fileName['SAROP11-PALMK118:CH2_BUFFER/pulse_id'][:]   
        
        try:
            ### will use the following lines will work if there is TIFALL5 in the channel list
            EventCode = fileName['SAR-CVME-TIFALL5:EvtSet/data']
            FEL = EventCode[:,48]
            Laser = EventCode[:,18]
            Darkshot = EventCode[:,21]
            index_ok = np.logical_and.reduce((FEL, Laser, np.logical_not(Darkshot)))
            print ('Shots selection done with event code')
        except:
            ### will use the following line if there is no TIFALL5, need to know the FEL/laser reprate however...
            mod = 2   # for 50 Hz
            index_ok = PulseIds%mod == 0          
            print ('Shots selection done with pulseID modulo = {}'.format(mod))
                
        tmpP = palm.etofs['1'].convert(input_data=TOF[index_ok], interp_energy=palm.energy_range)
        tmpUp = palm.etofs['0'].convert(input_data=uTOF[index_ok],interp_energy=palm.energy_range)
        
        eVIntP.append(tmpP)
        eVIntUp.append(tmpUp)
        wf_str.append(TOF[index_ok])
        wf_ref.append(uTOF[index_ok])
        clear_output(wait=True)
        print('Loaded file %s' %str(dataFiles['scan_files'][i][0]))


eVIntP = np.array(eVIntP)
eVIntUp = np.array(eVIntUp)

wf_str = np.array(wf_str)
wf_ref = np.array(wf_ref)
StagePOS = np.array(StagePOS)
Stagefs = (StagePOS[:,0]*1e-3*2/3e8)*1e15

Datasize = str(wf_str.shape)
print('Datasize = {}'.format(Datasize))

 ### Plot the THz scan to see whether the energy range is fine

In [ ]:
fig = plt.figure()
plt.pcolormesh(StagePOS[:,0], palm.energy_range, np.transpose(eVIntP.mean(axis=1)))
plt.xlabel('Delay [s]')
plt.ylabel('Electron energy [eV]')
plt.title(fn_json.split('/')[-1])
plt.show()

 ### Now calculate t0 and calibration factor with the fit

In [ ]:
### Load is done, now run this cell to calculate the calibration factor (eV2fs)
### For p18182 (5 keV) found -28.240766953955095 eV/fs
### For p18389 (12 keV) found -4.610191678065906 eV/fs

# determining t0 and peak-to-peak distance
streak=np.transpose(np.array([StagePOS[:,0], palm.energy_range[np.argmax(np.transpose(eVIntP.mean(axis=1)),axis =0)]]))
streakDerivative=np.transpose(np.array([[np.mean(t) for t in np.transpose([streak[:-1,0],streak[1:,0]])],np.diff(streak[:,1])/np.diff(streak[:,0])]))
#streakDerivative=np.transpose(np.array([streak[:-1,0],np.diff(streak[:,1])/np.diff(streak[:,0])]))
t0=[t[0] for t in streakDerivative if t[1]==min(streakDerivative[:,1])][0]

print("t0 is at {} s".format(t0))
print("Max streak is {} eV".format(max(np.array([t for t in streak if t[0]<t0])[:,1])-min(np.array([t for t in streak if t[0]>t0])[:,1])))

#Now fit with fit function
param_bounds = ([300,10,5.4e-10,1e-13,2e12,0],[600,500,5.5e-10,3e-12,4e12,np.inf])
parameters,extras = curve_fit(fitThz,StagePOS[:,0],palm.energy_range[np.argmax(np.transpose(eVIntP.mean(axis=1)),axis =0)], 
                              p0 = [np.min(palm.energy_range), 50, t0,2e-12,2.8e12,1])#,bounds=param_bounds)
# make the derivative 
dx = StagePOS[2,0]-StagePOS[1,0]
d_fitTHz_dx = np.gradient(fitThz(StagePOS[:,0],*parameters), dx)

max_y = np.max(np.gradient(fitThz(StagePOS[:,0],*parameters)))  # Find the maximum y value
max_x = np.argmin(np.gradient(fitThz(StagePOS[:,0],*parameters)))  # Find the maximum y value

# t0 from the phase fit 
t0_from_phase = parameters[2] + (parameters[4]*parameters[5])**-1 

calibrationLineSlope=np.interp(t0_from_phase,StagePOS[:,0],d_fitTHz_dx)
calibrationLineIntercept=np.interp(t0_from_phase,StagePOS[:,0],fitThz(StagePOS[:,0],*parameters))-calibrationLineSlope*t0_from_phase
ev2fsCalib = 1/(calibrationLineSlope*1e-15)
#print(max_x, max_y)
print ("t0 from phase is at {} s".format(t0_from_phase))
print ("Calibration factor is {} eV/fs".format(ev2fsCalib))
#print(parameters[2])
#print([calibrationLineSlope,calibrationLineIntercept,t0_from_phase])
print (calibrationLineIntercept)
# Trace from fit
xS=np.arange(np.min(StagePOS[:,0]),np.max(StagePOS[:,0]),(np.max(StagePOS[:,0])-np.min(StagePOS[:,0]))/300)
ySFitted=[fitThz(x, parameters[0],parameters[1],parameters[2],parameters[3],parameters[4],parameters[5]) for x in xS]

# Trace line 
#Trace_lin_fit = calibrationLineSlope * 


# Plot
fig = plt.figure(figsize=(8,4))
plt.pcolormesh(StagePOS[:,0], palm.energy_range, np.transpose(eVIntP.mean(axis=1)), cmap='CMRmap')
plt.colorbar()
plt.plot(xS,ySFitted)
plt.axvline(t0_from_phase)
plt.xlabel('Delay [s]')
plt.ylabel('Electron energy [eV]')
plt.title(scan_name)
plt.show()
PeakToPeak = np.max(ySFitted)-np.min(ySFitted)
print('Scan {}, t0 from phase = {} and peak-to-peak streak of {} eV'.format(scan_name, t0_from_phase, np.round(PeakToPeak,2)))

In [ ]:
DIR = "/sf/alvra/data/p18182/raw/scan_data/SiN_timingHe_3/"

!ls -lah -rt $DIR | grep BSREAD

!hostname

listfile = os.listdir(DIR)
number_files = len(listfile)
print ("There are",number_files," BSREAD files in the folder", DIR)

In [ ]:
# Analyse a single file, no dark / light selection of the shots!!

fff = DIR + "run_000397.BSREAD.h5"
#print (os.path.isfile(fileName))

with h5py.File(fff, 'r') as fileName:
    fileName = load_data._get_data(fileName)
    check = fileName["SAR-CVME-TIFALL5:EvtSet/is_data_present"][:]
    

#pulse_id, delays, pulse_lengths, debug_data = palm.process_hdf5_file(fileName, debug=True)

pulse_id, delays, _, (input_data, lags, cross_corr, _) = palm.process_hdf5_file(fff, debug=True)
ev2fsCalib = -3.5391553376634146
delays_fs = delays * ev2fsCalib

In [ ]:
# Check one shot (be careful, could be a dark one!!) to see the TOF peak and cross correlation

shot = 4

print ('Delay for shot {} = {} with calibration = {} eV/fs'.format(shot, delays[shot]*ev2fsCalib, ev2fsCalib))
print (np.amax(input_data['0'],axis=1)[shot])
print (cross_corr[shot].sum())

plt.figure(figsize= (10,5))
plt.subplot(121)
plt.plot(palm.energy_range, input_data['0'][shot], label="unstreaked")
plt.plot(palm.energy_range, input_data['1'][shot], label="streaked")
plt.legend(loc="best")

plt.subplot(122)
plt.plot(lags, cross_corr[shot], label="cross correlation")
plt.legend(loc="best")
plt.grid()

# Analyse a series of files

In [ ]:
datalist = glob.glob(DIR + "*.BSREAD.h5")
datalist = sorted(datalist)
datalist

## Next is to extract TT arrival times (choose FEL or laser as pump)

In [ ]:
datalist = glob.glob(DIR + "*.BSREAD.h5")
datalist = sorted(datalist)

ev2fsCalib= -20.723024835035712 #eV/fs    # This is from THz scan_508, p18182
#ev2fsCalib= -18.913301646032846 #eV/fs    # This is from THz scan_010, p18442
#ev2fsCalib = -3.5391553376634146 #eV/fs    # This is for 12 KeV

Delays_PALM_all = []
Delays_mean = []
cross_corr_all = []
str_OK = []
ref_OK = []
pulseID_filter = []
pulseID_first = []
run_number = []

nshots = None

for fileName in datalist:
    testdata = load_data.check_files_and_data(fileName)
    
    if (testdata):# and (i!=12):
        with h5py.File(fileName, 'r') as BS_file:
#            clear_output(wait=True)
            print ("Processing file {}". format(fileName))
            run_number.append(int(fileName.split('_')[-1].split('.')[0]))

            pulse_id_PALM, delays_PALM_eV, _, (input_data, lags, cross_corr, _) = \
                                                palm.process_hdf5_file(fileName, debug=True)
            
            pulseID_first.append(pulse_id_PALM[0]) 
                        
            #(reprate_light, reprate_dark), mod_out = load_data.load_reprates_laser_pump(fileName)
            (reprate_light, reprate_dark), mod_out = load_data.load_reprates_FEL_pump(fileName)
            
            pulse_id_PALM = pulse_id_PALM[:len(reprate_light)]
            delays_PALM_eV = delays_PALM_eV[:len(reprate_light)]
            cross_corr = cross_corr[:len(reprate_light)]
            input_data_0 = input_data['0'][:][:len(reprate_light)]
            input_data_1 = input_data['1'][:][:len(reprate_light)]
            
            delays_PALM_fs = delays_PALM_eV * ev2fsCalib
            
            fullArraySize = int(len(delays_PALM_fs)/mod_out)
            
            beamOK = (np.amax(input_data_0,axis=1) > 0.1)# & (delays !=0.0)

            delays_beamOK_light = delays_PALM_fs[beamOK & reprate_light]
            cross_corr_OK = cross_corr[beamOK & reprate_light]
            pulseID_OK = pulse_id_PALM[beamOK & reprate_light]
            refData_OK = input_data_0[beamOK & reprate_light]
            StrData_OK = input_data_1[beamOK & reprate_light]
        
            shots2pad = fullArraySize - len(delays_beamOK_light)
            print ('Modulo is {}, light shots are {}, need to pad {} shots'.format(mod_out, len(delays_beamOK_light), shots2pad))
            
            delays_beamOK_light = np.pad(delays_beamOK_light, (0, shots2pad), constant_values=np.NaN)
            
            Delays_PALM_all.append(delays_beamOK_light)
            cross_corr_all.append(cross_corr_OK)
            pulseID_filter.append(pulseID_OK)
            ref_OK.append(refData_OK)
            str_OK.append(StrData_OK)
        
        Delays_mean.append(np.nanmean(delays_beamOK_light))
            
        print ("Run {}, Pulse ID = {} ----- mean delay = {} fs".format(fileName.split('_')[-1].split('.')[0], \
                pulse_id_PALM[0], np.round(np.nanmean(delays_beamOK_light),3)))#, fileName,"pulseID =",pulse_id[0],)
        
Delays_PALM_all = np.asarray(Delays_PALM_all)
Delays_mean = np.asarray(Delays_mean)
cross_corr_all = np.asarray(cross_corr_all)
ref_OK = np.array(ref_OK)
str_OK = np.array(str_OK)
pulseID_filter = np.array(pulseID_filter)
#pulseID_first = np.array(pulseID_first)
print ("Job done!")
print ("Calibration factor used is {} eV/fs".format(ev2fsCalib))

In [ ]:
Delays_PALM_all.shape

In [ ]:
fig, ax1 = plt.subplots(figsize=(8,6))
plt.title(DIR)

plt.grid()
plt.xlabel('Run number')
ax1.plot(run_number, Delays_mean, label = 'Arrival time', marker ='.', color = 'r')
ax1.set_ylabel('mean arrival time (fs)', color='r')

ax2 = ax1.twinx()
for i in range(len(Delays_PALM_all)):
    xe = run_number[i]
    ye = len(Delays_PALM_all[i])
    ax2.plot(xe, ye, label = 'mean arrival time',marker = '.', color ='b')
ax2.set_ylabel('Number of shots per run', color='b')

plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.suptitle(DIR)

plt.subplot(121)
plt.hist((Delays_PALM_all[0,:]-np.nanmean(Delays_PALM_all)), bins =np.arange(-100,100,5), facecolor='blue', \
         label="sigma = {} fs".format(np.round(np.nanstd(Delays_PALM_all[0,:]-np.nanmean(Delays_PALM_all)), 3), alpha=0.5))
plt.title('single run')
plt.legend(loc="best")
plt.xlabel ('delay (fs)')
plt.show()

plt.subplot(122)
plt.hist((Delays_PALM_all.ravel()-np.nanmean(Delays_PALM_all)), bins =np.arange(-100,100,5), facecolor='blue', \
         label="sigma = {} fs".format(np.round(np.nanstd(Delays_PALM_all.ravel()-np.nanmean(Delays_PALM_all)), 3), alpha=0.5))
plt.title('all runs')
plt.legend(loc="best")
plt.xlabel ('delay (fs)')
plt.show()

In [ ]:
saveDir = "/das/work/p17/p17569/alvra_beamline_scripts/ProcessedData/"    # dest folder (needs to exist)
fn = str(saveDir + "Runs_451_513.h5")

hf = h5py.File(fn, 'w')
hf.create_dataset('Total_light_1D', data=Total_light_1D)
hf.create_dataset('Total_pulseIDS', data=Total_pulseIDS)
hf.create_dataset('Total_strData', data=Total_strData)
hf.create_dataset('Total_refData', data=Total_refData)
hf.create_dataset('Total_corrData', data=Total_corrData)
hf.close()

In [ ]:
itemindex = np.where(Total_light_1D==66.05130091)

In [ ]:
import pandas as pd
#input_data_max = np.amax(input_data['0'],axis=1)
df = pd.DataFrame(Total_light_1D)
Avg = df.rolling(300).mean()

In [ ]:
plt.figure(figsize= (10,5))
#plt.plot(Total_light_1D[0::1000])
plt.plot(Total_pulseIDS, Total_light_1D[:],'.')
#plt.plot(Total_light_1D[:],'.')

plt.plot(Total_pulseIDS, Avg,'.', ms=1 )


for xc in pulseID_first:
    if run_number[pulseID_first.index(xc)]%5 ==0:
        plt.axvline(x=xc, color = 'r')
        #plt.text(xc,200, pulseID_first.index(xc)+start_index,color = 'r')
        plt.text(xc,350, run_number[pulseID_first.index(xc)],color = 'r')

plt.ylabel("Arrival time (fs)")
#plt.xlabel("shot number")
plt.xlabel("Pulse ID")

plt.ticklabel_format(useOffset=False, style='plain')
plt.ylim((-480,400))
plt.grid()
